In [1]:
import json
import pandas as pd

In [2]:
filename = "/home/asureddy_umass_edu/cs682/VILA/results/keypoint/VILA1.5-13b_8-shot.json"
with open(filename,'rb') as f:
    data = json.load(f)

In [2]:
def compute_precision_recall_fscore(true_positive, false_positive, false_negative):
    """
    Compute precision, recall, and F1-score.

    Args:
        true_positive (int): Number of true positives.
        false_positive (int): Number of false positives.
        false_negative (int): Number of false negatives.

    Returns:
        tuple: (precision, recall, f1_score)
    """
    # Precision: TP / (TP + FP)
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0.0

    # Recall: TP / (TP + FN)
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0.0

    # F1-Score: 2 * (precision * recall) / (precision + recall)
    f1_score = (
        2 * (precision * recall) / (precision + recall)
        if (precision + recall) > 0
        else 0.0
    )

    return precision, recall, f1_score

# Example usage
true_positive = 50
false_positive = 10
false_negative = 5

precision, recall, f1_score = compute_precision_recall_fscore(
    true_positive, false_positive, false_negative
)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1_score:.2f}")


Precision: 0.83
Recall: 0.91
F1-Score: 0.87


In [3]:
def compute_kpp_scores(data_file):
    with open(data_file,'rb') as f:
        data = json.load(f)
    vis = [x['output'].strip().lower() for x in data["yes"]]
    not_vis = [x['output'].strip().lower() for x in data["no"]]
    tp = sum([1 for x in vis if x=="visible"])
    fn = len(vis)-tp
    tn = sum([1 for x in not_vis if x=="not visible"])
    fp = len(vis)-tn
    p,r,f = compute_precision_recall_fscore(tp,fp,fn)
    a = (tp+tn)/(fp+fn)
    return {
        "precision": p,
        "recall": r,
        "f-score": f,
        "accuracy": a
    }

In [5]:
compute_kpp_scores(filename)

{'precision': 0.41331658291457285,
 'recall': 0.491044776119403,
 'f-score': 0.4488403819918145,
 'accuracy': 0.6584158415841584}

### Generating Csvs

In [4]:
def generate_keypoint_metrics_csv(res_file_template, n_shots, output_csv_path):
    # Initialize a list to store results
    results = []

    # Iterate through the number of shots and calculate metrics
    for n in n_shots:
        res_file = res_file_template.format(n_shots=n)
        metrics = compute_kpp_scores(res_file)
        metrics['n_shots'] = n
        results.append(metrics)

    # Create a pandas DataFrame from the results
    df = pd.DataFrame(results)

    # Save the results to a CSV file
    df.to_csv(output_csv_path, index=False)

    print(f"Results saved to {output_csv_path}")


In [5]:
# fixed params
n_shots = [0, 2, 4, 8]
model_names = ["VILA1.5-3b", "VILA1.5-13b"]
strategy_names = ["random"]
dataset_names = ["coco"]

In [6]:
# Parameters
for dataset_name in dataset_names:
    for model_name in model_names:
        for strategy in strategy_names:
            if dataset_name=="coco":
                dataset_str = "/home/asureddy_umass_edu/cs682/VILA_codes/results/keypoint/"
            
            res_file_template = dataset_str+str(model_name)+"_{n_shots}-shot.json"
            op_path = f"/home/asureddy_umass_edu/cs682/metrics_results/keypoint_presence/{dataset_name}_{model_name}_{strategy}.csv"
            generate_keypoint_metrics_csv(res_file_template, n_shots, op_path)


Results saved to /home/asureddy_umass_edu/cs682/metrics_results/keypoint_presence/coco_VILA1.5-3b_random.csv
Results saved to /home/asureddy_umass_edu/cs682/metrics_results/keypoint_presence/coco_VILA1.5-13b_random.csv


### Visually

In [6]:
print("Metrics for a 3B model")
n_shots = [0,2,4,8]
for shot in n_shots:
    print(f"n-shots: {shot}")
    f_name = f"/home/asureddy_umass_edu/cs682/VILA/results/keypoint/VILA1.5-3b_{shot}-shot.json"
    print(compute_kpp_scores(f_name))

Metrics for a 3B model
n-shots: 0
{'precision': 0.0, 'recall': 0.0, 'f-score': 0.0, 'accuracy': 0.0}
n-shots: 2
{'precision': 0.0, 'recall': 0.0, 'f-score': 0.0, 'accuracy': 0.3604060913705584}
n-shots: 4
{'precision': 0.0, 'recall': 0.0, 'f-score': 0.0, 'accuracy': 0.027607361963190184}
n-shots: 8
{'precision': 0.0, 'recall': 0.0, 'f-score': 0.0, 'accuracy': 0.006006006006006006}


In [7]:
print("Metrics for a 13B model")
n_shots = [0,2,4,8]
for shot in n_shots:
    print(f"n-shots: {shot}")
    f_name = f"/home/asureddy_umass_edu/cs682/VILA/results/keypoint/VILA1.5-13b_{shot}-shot.json"
    print(compute_kpp_scores(f_name))

Metrics for a 13B model
n-shots: 0
{'precision': 0.1783876500857633, 'recall': 0.15522388059701492, 'f-score': 0.16600159616919394, 'accuracy': 0.2822966507177033}
n-shots: 2
{'precision': 0.16304347826086957, 'recall': 0.08955223880597014, 'f-score': 0.11560693641618498, 'accuracy': 0.4596949891067538}
n-shots: 4
{'precision': 0.1309192200557103, 'recall': 0.07014925373134329, 'f-score': 0.0913508260447036, 'accuracy': 0.43315508021390375}
n-shots: 8
{'precision': 0.41331658291457285, 'recall': 0.491044776119403, 'f-score': 0.4488403819918145, 'accuracy': 0.6584158415841584}
